In [1]:
import os 
import pandas as pd
import re
import numpy as np

from sklearn import preprocessing
from sklearn import decomposition 

/Users/edwardyao/anaconda3/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#os.listdir()
PATH = os.getcwd()

In [3]:
# load 2001 and 2011 data
census_raw_01 = pd.read_csv('Census2001.csv')
census_raw_11 = pd.read_csv('Census2011.csv')

# drop nan rows
census_raw_01.dropna(inplace=True)
census_raw_11.dropna(inplace=True)

#display(census_raw_01)
print(f'Num rows in 2001: {len(census_raw_01)} \nNum rows in 2011: {len(census_raw_11)} \n')

Num rows in 2001: 1609 
Num rows in 2011: 1673 



In [4]:
# subset to the columns used for scoring
def extract(df, verbose=False):
    
    pattern = "(LSOA)|(Pincode)|(.*income.*)|(.+occupation$)|(^[1-3]?\..+)|(.+16-74$)|(.*level.*4.*)"
    m = re.compile(pattern, re.I)
    l = [c for c in df.columns if m.match(c)]
    d = df[l]
    
    # change df to numeric
    d.columns = map(str.lower, d.columns)
    num_cols = [c for c in d.columns if c not in ['lsoa', 'pincode']]
    d[num_cols] = d[num_cols].apply(pd.to_numeric)
    
    if verbose:
        print(f'd:\n{d.dtypes}\n')

    occ_cols = list(d.filter(regex='^[1-3]?\..+').columns)
    
    d['sum_occ'] = d.filter(regex='^[1-3]?\..+').sum(axis=1)
    d['total_occ'] = d.filter(regex='.+categories.*')
    d['qual'] = d.filter(regex='.*level.*4.*') 
    d['total_qual'] =  d.filter(regex='.+16-74$')
    
    d['occupation'] = d['sum_occ'] / d['total_occ']
    d['qualification'] = d['qual'] / d['total_qual']
    d['log_med_house_price'] = np.log(d.filter(regex='.*income.*'))

    cols = ['lsoa', 'pincode', 'occupation', 'qualification', 'log_med_house_price']

    d = d.filter(cols)
    
    if verbose:
        print(f'Dataframe after processing:\n{d.dtypes}')
    
    assert len(d) == len(df), 'Out df has wrong num rows...'
    
    return d

In Column="Median Household Income" there is a ":" in 36 rows of 2001 census and 100 rows of 2011 census

In [5]:
# further cleaning df, there was issue with median household income datatype

invalid_rows_01 = census_raw_01[census_raw_01['Median Household income'] == ':'].index.values.tolist()
census_raw_01_clean = census_raw_01.drop(index=invalid_rows_01)

invalid_rows_11 = census_raw_11[census_raw_11['Median Household income'] == ':'].index.values.tolist()
census_raw_11_clean = census_raw_11.drop(index=invalid_rows_11)
c = census_raw_11_clean.loc[:, 'Median Household income']
c = c.str.split(',').str.join('').astype(float)
#c.name = 'Median Household income'
census_raw_11_clean.drop('Median Household income', axis=1, inplace=True)
census_raw_11_clean.insert(len(census_raw_11.columns)-1, 'Median Household income', list(c))

In [6]:
# scale and pca
def scale_pca(df1, df2, verbose=False):
    
    c01 = extract(df1)
    c11 = extract(df2)
    
    c01 = c01.iloc[:, 1:]
    c11 = c11.iloc[:, 1:]
    
    # turn pincode into index
    c01.set_index('pincode', inplace=True)
    c11.set_index('pincode', inplace=True)
    
    common_index = c01.merge(c11, how='inner', on='pincode').index
    
    # use c01 index to filter c11 rows
    c01 = c01.loc[common_index]
    c11 = c11.loc[common_index]
    
    # scale data
    stacked = np.concatenate((c01.values, c11.values), axis=0)
    
    scale = preprocessing.RobustScaler().fit(stacked)
    
    data_scaled = scale.transform(stacked)
    
    # pca
    pca = decomposition.PCA(n_components=1)
    pca.fit(data_scaled)            
    scores_T = pd.DataFrame(pca.transform(stacked))
    
    
    # attach scores back to original dfs
    score_01 = scores_T.loc[: len(c01)-1, 0]
    score_11 = scores_T.loc[len(c01): , 0]
    
    print(len(c01))
    print(len(score_01))
    print(len(score_11))

    c01 = c01.assign(score=pd.Series(score_01).values)
    c11 = c11.assign(score=pd.Series(score_11).values)
    
    if verbose:
        print(f'c01 num rows: {len(c01)}\nc01:\n{c01.head(5)}\n')
        print(f'c11 num rows: {len(c11)}\nc01:\n{c11.head(5)}\n')
    
    # merge and calculate additional statistics from the scores
    combined = c01.merge(c11, how='inner', on='pincode', suffixes=('_01', '_11'))
    
    # 2001 lsoa rank
    combined['rank_01'] = combined.score_01.rank(ascending=False)

    # 2011 lsoa rank
    combined['rank_11'] = combined.score_11.rank(ascending=False)

    # 2001 to 2011 score increase
    combined.loc[:,'ses_asc'] = combined.loc[:,'score_11'] - combined.loc[:,'score_01']
    
    # 2001 LSOA percentile score
    combined.loc[:,'score_pr_01'] = combined.rank_01.rank(ascending=False, pct=True) * 100

    # 2011 LSOA percentile score
    combined.loc[:,'score_pr_11'] = combined.score_11.rank(ascending=False, pct=True) * 100

    # Calculate percentile change
    combined.loc[:,'score_pr_asc'] = combined.loc[:,'score_pr_11'] - combined.loc[:,'score_pr_01']
    
    if verbose:
        print(f'combined:\n{combined.dtypes}\n')
    
    return combined
    

In [7]:
# write to csv
test = scale_pca(census_raw_01_clean, census_raw_11_clean)
test.to_csv('scores.csv')

1510
1510
1510


/var/folders/47/vfwtfrrs15l9jrx68sw2gdww0000gn/T/ipykernel_54094/1874351655.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d[num_cols] = d[num_cols].apply(pd.to_numeric)
/var/folders/47/vfwtfrrs15l9jrx68sw2gdww0000gn/T/ipykernel_54094/1874351655.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  d['sum_occ'] = d.filter(regex='^[1-3]?\..+').sum(axis=1)
/var/folders/47/vfwtfrrs15l9jrx68sw2gdww0000gn/T/ipykernel_54094/1874351655.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a